# Example for utilization of powerfactory-tools -- Control

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import logging
import pathlib
import datetime as dt

from powerfactory_tools import PowerFactoryInterface
from powerfactory_tools.utils.io import to_json

In [ ]:
PROJECT_NAME = "PowerFactory-Tools"  # may be also full path "dir_name\project_name"
EXPORT_PATH = pathlib.Path("control_action_results")
PF_USER_PROFILE = ""  # specification may be necessary
PF_INI_NAME = ""  # optional specification of ini file name to switch to full version (e.g. PowerFactoryFull for file PowerFactoryFull.ini)
PYTHON_VERSION = "3.10"

In [ ]:
def export_data(
    data: dict,
    export_data_name: str | None,
    export_path: pathlib.Path,
) -> None:
    """Export data to json file.

    Arguments:
        data {dict} -- data to export
        export_data_name {str | None} -- the chosen file name for data
        export_path {pathlib.Path} -- the directory where the exported json file is saved
    """
    timestamp = dt.datetime.now().astimezone()  # noqa: DTZ005
    timestamp_string = timestamp.isoformat(sep="T", timespec="seconds").replace(":", "")
    if export_data_name is None:
        filename = f"{PROJECT_NAME}_{timestamp_string}.json"
    else:
        filename = f"{export_data_name}.json"

    file_path = export_path / filename
    try:
        file_path.resolve()
    except OSError as e:
        msg = f"File path {file_path} is not a valid path."
        raise FileNotFoundError(msg) from e

    to_json(data=data, file_path=file_path)

## Control using a controller instance

In [ ]:
export_data_name = "export_data"
grid_name = "HV_9_Bus"

with PowerFactoryInterface(
    project_name=PROJECT_NAME,
    powerfactory_user_profile=PF_USER_PROFILE,
    powerfactory_ini_name=PF_INI_NAME,
    python_version=PYTHON_VERSION,
    logging_level=logging.INFO,
) as pfi:
    ############
    ## Example 1: Request elements
    # Activate study case "Base"
    pfi.switch_study_case("Base")
    # Get grid object with specified name
    grid = pfi.grid(grid_name)
    # Get all objects from specific grid
    data = pfi.compile_powerfactory_data(grid)
    # All nodes in grid {grid_name}, also these that are out of service
    terminals_grid = data.terminals

    ############
    ## Example 2: Select special elements
    # All nodes of active study case which are in service and therefore calculation relevant
    terminals_study_case = pfi.terminals(calc_relevant=True)

    # Select only terminals with nominal voltage of xx kV
    voltage_threshold = 110
    terminals_sel = []  # selected terminals
    for term in terminals_study_case:
        # nominal voltage
        u_n = term.uknom
        if u_n == voltage_threshold:
            terminals_sel.append(term)

    ############
    ## Example 3: Change attribute values
    # Select some loads that names start with "Load"
    loads = pfi.loads("Load*", grid_name=grid_name)

    # Change power of loads
    for load in loads:
        load.plini = 2  # active power in MW

    ############
    ## Example 4: Run load flow - sym or unsym
    ldf_result = pfi.run_ldf(ac=True, symmetrical=False)
    # Do further user specific work and fill result_data dictionary based on PF result
    result_data = {}

    # Store results
    export_data(
        data=result_data,
        export_data_name=export_data_name,
        export_path=EXPORT_PATH,
    )

    ############
    ## Example 5: Request and change study cases, operation scenarios and network variantions
    # Study Case
    study_cases = pfi.study_cases()  # get all
    study_case_active = pfi.study_case(only_active=True)  # get only active one

    # Network Variation
    variations = pfi.grid_variants()  # get all
    variations_active = pfi.grid_variants(only_active=True)  # get only active one(s)

    # Operation Scenarios
    scenarios = pfi.scenarios()  # get all
    scenario_active = pfi.scenario(only_active=True)  # get only active one

    # Create Grid Variant
    pfi.switch_study_case("Industry_Park")

    variation_folder = pfi.create_folder(name="user_variations", location=pfi.grid_variant_dir, update=True)
    pfi.create_grid_variant(name="Variant1", location=variation_folder, update=True)

    # Set transformer out of service
    transformer = pfi.transformer_2w("Transformer_2w_110/20", grid_name=grid_name)
    transformer.outserv = 1

    # Deactivate all grid variants
    for variation in pfi.grid_variants(only_active=True):
        pfi.deactivate_grid_variant(variation)

    # Set only one grid active
    pfi.deactivate_grids()
    pfi.activate_grid(pfi.grid(grid_name))

## Control using the control function running in a new process with default parameters

In [ ]:
import multiprocessing

POWERFACTORY_PATH = pathlib.Path("C:/Program Files/DIgSILENT")
POWERFACTORY_VERSION = "2022 SP2"


class PowerFactoryControllerProcess(multiprocessing.Process):
    def __init__(
        self,
        *,
        export_path: pathlib.Path,
        project_name: str,
        grid_name: str,
        export_data_name: str = "",
        powerfactory_user_profile: str = "",
        powerfactory_path: pathlib.Path = POWERFACTORY_PATH,
        powerfactory_version: str = POWERFACTORY_VERSION,
        powerfactory_ini_name: str = "",
        python_version: str = PYTHON_VERSION,
        logging_level: int = logging.INFO,
    ) -> None:
        super().__init__()
        self.export_path = export_path
        self.project_name = project_name
        self.grid_name = grid_name
        self.export_data_name = export_data_name
        self.powerfactory_user_profile = powerfactory_user_profile
        self.powerfactory_path = powerfactory_path
        self.powerfactory_version = powerfactory_version
        self.powerfactory_ini_name = powerfactory_ini_name
        self.python_version = python_version
        self.logging_level = logging_level

    def run(self) -> None:
        pfi = PowerFactoryInterface(
            project_name=self.project_name,
            powerfactory_user_profile=self.powerfactory_user_profile,
            powerfactory_path=self.powerfactory_path,
            powerfactory_version=self.powerfactory_version,
            powerfactory_ini_name=self.powerfactory_ini_name,
            python_version=self.python_version,
            logging_level=self.logging_level,
        )
        ## Example 1: Request elements
        # Activate study case "Base"
        pfi.switch_study_case("Base")
        # Get grid object with specified name
        grid = pfi.grid(self.grid_name)
        # Get all objects from specific grid
        data = pfi.compile_powerfactory_data(grid)
        # All nodes in grid {grid_name}, also these that are out of service
        terminals_grid = data.terminals

        ############
        ## Example 2: Select special elements
        # All nodes of active study case which are in service and therefore calculation relevant
        terminals_study_case = pfi.terminals(calc_relevant=True)

        # Select only terminals with nominal voltage of xx kV
        voltage_threshold = 110
        terminals_sel = []  # selected terminals
        for term in terminals_study_case:
            # nominal voltage
            u_n = term.uknom
            if u_n == voltage_threshold:
                terminals_sel.append(term)

        ############
        ## Example 3: Change attribute values
        # Select some loads that names start with "Load"
        loads = pfi.loads(name="Load*", grid_name=self.grid_name)

        # Change power of loads
        for load in loads:
            load.plini = 2  # active power in MW

        ############
        ## Example 4: Run load flow - sym or unsym
        ldf_result = pfi.run_ldf(ac=True, symmetrical=False)
        # Do further user specific work and fill result_data dictionary based on PF result
        result_data = {}

        # Store results
        export_data(
            data=result_data,
            export_data_name=export_data_name,
            export_path=EXPORT_PATH,
        )

        ############
        ## Example 5: Request and change study cases, operation scenarios and network variantions
        # Study Case
        study_cases = pfi.study_cases()  # get all
        study_case_active = pfi.study_case(only_active=True)  # get only active one

        # Network Variation
        variation = pfi.grid_variants()  # get all
        variations_active = pfi.grid_variants(only_active=True)  # get only active one(s)

        # Operation Scenarios
        scenarios = pfi.scenarios()  # get all
        scenario_active = pfi.scenario(only_active=True)  # get only active one

        # Create Grid Variant
        pfi.switch_study_case("Industry_Park")

        variation_folder = pfi.create_folder(name="user_variations", location=pfi.grid_variant_dir, update=True)
        pfi.create_grid_variant(name="Variant1", location=variation_folder, update=True)

        # Set transformer out of service
        transformer = pfi.transformer_2w("Transformer_2w_110/20", grid_name=self.grid_name)
        transformer.outserv = 1

        # Deactivate all grid variants
        for variation in pfi.grid_variants(only_active=True):
            pfi.deactivate_grid_variant(variation)

        # Set only one grid active
        pfi.deactivate_grids()
        pfi.activate_grid(pfi.grid(self.grid_name))

In [ ]:
def apply_control(  # noqa: PLR0913
    export_path: pathlib.Path,
    project_name: str,
    grid_name: str,
    export_data_name: str = "",
    powerfactory_user_profile: str = "",
    powerfactory_path: pathlib.Path = POWERFACTORY_PATH,
    powerfactory_version: str = POWERFACTORY_VERSION,
    powerfactory_ini_name: str = "",
    python_version: str = PYTHON_VERSION,
    logging_level: int = logging.INFO,
) -> None:
    process = PowerFactoryControllerProcess(
        project_name=project_name,
        export_path=export_path,
        grid_name=grid_name,
        export_data_name=export_data_name,
        powerfactory_user_profile=powerfactory_user_profile,
        powerfactory_path=powerfactory_path,
        powerfactory_version=powerfactory_version,
        powerfactory_ini_name=powerfactory_ini_name,
        python_version=python_version,
        logging_level=logging_level,
    )
    process.start()
    process.join()

### As the control function is executed in a process that is terminated after execution, the PowerFactory API is also closed.

In [ ]:
export_data_name = "export_data"
grid_name = "HV_9_Bus"

apply_control(
    export_path=EXPORT_PATH,
    project_name=PROJECT_NAME,
    grid_name=grid_name,
    export_data_name=export_data_name,
    powerfactory_user_profile=PF_USER_PROFILE,
    powerfactory_ini_name=PF_INI_NAME,
    python_version=PYTHON_VERSION,
)